In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
img_path = os.listdir('/content/drive/MyDrive/brain/dataset')
imgs = []
root_dir = '/content/drive/MyDrive/brain/dataset/'

for path in img_path:
  img_path = tf.keras.preprocessing.image.load_img(
      root_dir+path,
      target_size=(224,224),
      color_mode = 'grayscale'
  )
  img_array = tf.keras.preprocessing.image.img_to_array(img_path)
  img_array /= 255
  imgs.append(img_array)

In [65]:
len(imgs)

100

In [66]:
imgs = np.array(imgs)

#Add Noise

In [67]:
noise_img = []
for img in imgs:
  noise = img + 0.2*np.random.randn(*img.shape)
  noise = np.clip(noise, 0, 1) # make range between 0 and 1
  noise_img.append(noise)

In [68]:
noise_img = np.array(noise_img)

In [69]:
noise_train = noise_img[:80]
noise_test = noise_img[80:]

normal_train = imgs[:80]
normal_test = imgs[80:]

noise_train.shape, noise_test.shape,normal_train.shape, normal_test.shape

((80, 224, 224, 1), (20, 224, 224, 1), (80, 224, 224, 1), (20, 224, 224, 1))

#Auto Encoder

In [70]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model, Sequential, load_model

def autoEncoder():
  input = Input((224, 224, 1))

  #encoder
  output = Conv2D(32, (3,3), activation='relu', padding='same')(input)
  output = MaxPooling2D((2,2), padding='same')(output)
  output = Conv2D(64, (3,3), activation='relu', padding='same')(output)
  output = MaxPooling2D((2,2), padding='same')(output)

  #decoder
  output = Conv2D(64, (3,3), activation='relu', padding='same')(output)
  output = UpSampling2D((2,2))(output)
  output = Conv2D(32, (3,3), activation='relu', padding='same')(output)
  output = UpSampling2D((2,2))(output)

  #output
  output = Conv2D(1, (3,3), activation='sigmoid', padding='same')(output)

  return Model(input, output)

In [71]:
autoencoder = autoEncoder()
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 224, 224, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 224, 224, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 56, 56, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_9 (UpSampling2D)       │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 112, 112, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_10 (UpSampling2D)      │ (None, 224, 224, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_27 (Conv2D)                   │ (None, 224, 224, 1)         │             289 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 74,497 (291.00 KB)

 Trainable params: 74,497 (291.00 KB)

 Non-trainable params: 0 (0.00 B)

In [72]:
autoencoder.fit(noise_train, normal_train, epochs=50, batch_size=16, validation_data=(noise_test, normal_test))

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 0.6474 - val_loss: 0.5399
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - loss: 0.5371 - val_loss: 0.5085
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 0.4845 - val_loss: 0.4448
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 0.4274 - val_loss: 0.4236
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 0.4208 - val_loss: 0.4239
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 0.4079 - val_loss: 0.4105
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 0.3983 - val_loss: 0.4067
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 0.3939 - val_loss: 0.4036
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 0.3907 - val_loss: 0.4026
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 0.3933 - val_loss: 0.4006
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 0.3917 - val_loss: 0.3987
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 0.3850 - val_loss: 0.3962
Epoch 13/50
5/5

In [73]:
predict = autoencoder.predict(noise_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [74]:
from sklearn.metrics import mean_squared_error
predict = predict.flatten()
normal_test = normal_test.flatten()
mse = mean_squared_error(predict, normal_test)
print(mse)

0.011857727


In [75]:
from skimage.metrics import peak_signal_noise_ratio
# Calculate PSNR for each pair of clean and denoised images and take the absolute value
psnr_val = [abs(peak_signal_noise_ratio(Normal,deNoise,data_range=deNoise.max()-deNoise.min())) for Normal,deNoise in zip(normal_test, predict)]
psnr = np.mean(psnr_val)
print(f"psnr_val -> {psnr}")

psnr_val -> inf
